<img style="float: right;" src="hyperstream.svg">

# HyperStream Tutorial 4: Real-time streams

We will be ussing the tool created in the previous tutorial and we will compose the output of the stream with a new one.

In [1]:
%load_ext watermark

import sys
from datetime import datetime
from datetime import datetime, timedelta

sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream
from hyperstream import TimeInterval
from hyperstream.utils import UTC

%watermark -v -m -p hyperstream -g

CPython 2.7.6
IPython 5.3.0

hyperstream 0.3.0-beta

compiler   : GCC 4.8.4
system     : Linux
release    : 3.19.0-80-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : 33621000d8f335505eac32a5a274194d2588a4c9


In [2]:
hs = HyperStream(loglevel=20)
print hs

environment_stream = hs.channel_manager.memory.get_or_create_stream("environment")

now = datetime.utcnow().replace(tzinfo=UTC)
before = (now - timedelta(minutes=30)).replace(tzinfo=UTC)

ti = TimeInterval(before, now)

environment_tool = hs.plugins.example.tools.environment_data_gov_uk()

environment_tool.execute(sources=[], sink=environment_stream, interval=ti)

for key, value in environment_stream.window().items():
    print '[%s]: %s' % (key, value)

HyperStream version 0.3.0-beta, connected to mongodb://localhost:27017/hyperstream
[2017-07-04 08:39:45+00:00]: [{'value': 9.428, 'measure': u'E24808-level-stage-i-15_min-mAOD'}]
[2017-07-04 08:40:00+00:00]: [{'value': 0.264, 'measure': u'E22561-flow-stage-i-15_min-m3_s'}]
[2017-07-04 08:45:00+00:00]: [{'value': 14.057, 'measure': u'2900TH-flow--Mean-15_min-m3_s'}, {'value': 0.03, 'measure': u'E21863-flow-stage-i-15_min-m3_s'}, {'value': 0.71, 'measure': u'E15611-level-stage-i-15_min-mASD'}, {'value': -0.433, 'measure': u'BB_EAST_UR-level-tidal_level-i-15_min-mAOD'}, {'value': 0.564, 'measure': u'53120-level-stage-i-15_min-m'}, {'value': 0.076, 'measure': u'E21187-level-stage-i-15_min-m'}, {'value': 7.254, 'measure': u'DIFF_TB3_TB9-level-tidal_level-i-15_min-m'}, {'value': 0.041, 'measure': u'DIFF_TB2_TB9-level-tidal_level-i-15_min-m'}, {'value': 2.386, 'measure': u'E70139-north_west-level-tidal_level-Mean-15_min-mAOD'}, {'value': 2.52, 'measure': u'E73439-north_west-level-tidal_level-